Thoushands or even millions of features not only make training extremely slow, also make it harder to find a good solution, this is called **the curse of dimensionality**.

# Main content
- the curse of dimensionality
- know high-dimensional space
- projection and Manifold learning
- PCA, Kernel PCA, LLE.

# The Curse of Dimensionality
The more dimensions the training set has, the greater the risk of overfitting it. In theory, one solution to the curse of dimensionality could be to increase the size of training set to reach a sufficient density of training instances.

# Main Approaches